<a href="https://colab.research.google.com/github/learner-NKH/Weighted-Boxes-Fusion/blob/master/WBF_GHS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

In [ ]:
# !pip install kaggle
# from google.colab import files
# files.upload()
# #出现添加框后选择kaggle.json文件上传
# !mkdir -p ~/.kaggle 
# !cp kaggle.json ~/.kaggle/ 
# !chmod 600 ~/.kaggle/kaggle.json 
# !kaggle datasets download -d xthink/coco-2017-val-images -p '../content/drive/My Drive'
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install wandb >/dev/null 2>&1

In [ ]:
!wandb login 214404d703d3f8fbcbec6951fa4cbb717d03b233

In [ ]:
!cp -r ./drive/MyDrive/dwbf/wbf_output/ ./wbf_output

In [ ]:
!mkdir ./data
# !cp ./drive/MyDrive/coco-2017-val-images.zip ./data/coco-2017-val-images.zip

In [ ]:
# !unzip -d ./data/coco_val2017 ./data/coco-2017-val-images.zip
# !rm ./data/coco-2017-val-images.zip

In [ ]:
!cp -r ./drive/MyDrive/dwbf/ensemble_boxes-1.0.4nkh/ ./ensemble_boxes-1.0.4nkh

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

In [ ]:
!mkdir ./data/coco
!unzip -d ./data/coco/ annotations_trainval2017.zip

# Install libriary

In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html >/dev/null 2>&1
!git clone https://github.com/open-mmlab/mmdetection.git >/dev/null 2>&1

%cd mmdetection
!pip install -r requirements/build.txt >/dev/null 2>&1
!pip install -v -e . >/dev/null 2>&1  # or "python setup.py develop"
%cd ..

!git clone -b mmdetection https://github.com/Chien-Hung/DetVisGUI.git >/dev/null 2>&1

In [ ]:
import mmcv
from mmcv import Config, DictAction

import pandas as pd
import numpy as np
import random
import sys
sys.path.append('./mmdetection')
import mmdet
from mmdet.datasets import build_dataset
from mmdet.utils import update_data_root
import cv2
from tqdm.notebook import tqdm
tqdm.pandas()
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import glob
import shutil
import torch
from PIL import Image
import ast

In [ ]:
sys.path.append('/content/DetVisGUI')
# sys.path.append('./ensemble_boxes-1.0.4nkh')

In [ ]:
import warnings
import numpy as np
from numba import jit
import random

# Functions

In [ ]:
def run_nms(bboxes, confs,classs, image_size, iou_thr=0.50, skip_box_thr=0.0001, weights=None):
    boxes =  [bbox/(image_size-1) for bbox in bboxes]
    scores = [conf for conf in confs]
    labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = nms(boxes, scores, labels, weights=weights, iou_thr=iou_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

def run_nmw(bboxes, confs,classs, image_size, iou_thr=0.50, skip_box_thr=0.0001, weights=None):
    boxes =  [bbox/(image_size-1) for bbox in bboxes]
    scores = [conf for conf in confs]
    labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = non_maximum_weighted(boxes, scores, labels, weights=weights, iou_thr=iou_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels


def run_soft_nms(bboxes, confs,classs, image_size, iou_thr=0.50, skip_box_thr=0.0001,sigma=0.1, weights=None):
    boxes =  [bbox/(image_size-1) for bbox in bboxes]
    scores = [conf for conf in confs]    
    labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = soft_nms(boxes, scores, labels, weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

def run_wbf(bboxes, confs, classs, image_size, iou_thr=0.50, skip_box_thr=0.0001, weights=None):
    boxes =  [bbox/(image_size-1) for bbox in bboxes]
    scores = [conf for conf in confs]    
    labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels, num_dui, num_dui2 = weighted_boxes_fusion(boxes, scores, labels, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels, num_dui, num_dui2

In [ ]:
def voc2yolo(bboxes, image_height=720, image_width=1280):
    """
    voc  => [x1, y1, x2, y1]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]/ image_height
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    bboxes[..., 0] = bboxes[..., 0] + w/2
    bboxes[..., 1] = bboxes[..., 1] + h/2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    return bboxes

def yolo2voc(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    return bboxes

def coco2yolo(bboxes, image_height=720, image_width=1280):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    return bboxes

def yolo2coco(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    # denormalizing
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    # converstion (xmid, ymid) => (xmin, ymin)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    return bboxes

def voc2coco(bboxes, image_height=720, image_width=1280):
    bboxes  = voc2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2coco(bboxes, image_height, image_width)
    return bboxes

def coco2voc(bboxes, image_height=720, image_width=1280):
    bboxes  = coco2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2voc(bboxes, image_height, image_width)
    return bboxes

def load_image(image_path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [255, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def draw_bboxes(img, bboxes, classes, class_ids, colors = None, show_classes = None, bbox_format = 'yolo', class_name = False, line_thickness = 2):

    image = img.copy()
    show_classes = classes if show_classes is None else show_classes
    colors = (0, 255 ,0) if colors is None else colors

    if bbox_format == 'yolo':

        for idx in range(len(bboxes)):

            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:

                x1 = round(float(bbox[0])*image.shape[1])
                y1 = round(float(bbox[1])*image.shape[0])
                w  = round(float(bbox[2])*image.shape[1]/2) #w/2
                h  = round(float(bbox[3])*image.shape[0]/2)

                voc_bbox = (x1-w, y1-h, x1+w, y1+h)
                plot_one_box(voc_bbox,
                             image,
                             color = color,
                             label = cls if class_name else str(get_label(cls)),
                             line_thickness = line_thickness)

    elif bbox_format == 'coco':

        for idx in range(len(bboxes)):

            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                w  = int(round(bbox[2]))
                h  = int(round(bbox[3]))

                voc_bbox = (x1, y1, x1+w, y1+h)
                plot_one_box(voc_bbox,
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)

    elif bbox_format == 'voc_pascal':

        for idx in range(len(bboxes)):

            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                x2 = int(round(bbox[2]))
                y2 = int(round(bbox[3]))
                voc_bbox = (x1, y1, x2, y2)
                plot_one_box(voc_bbox,
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)
    else:
        raise ValueError('wrong bbox format')

    return image

def show_img(img, bboxes, bbox_format='yolo', bbox_colors = None):
    names  = ['starfish']*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes,
                           classes = names,
                           class_ids = labels,
                           class_name = True,
                           colors = colors if bbox_colors is None else bbox_colors,
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return Image.fromarray(img).resize((800, 400))

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

In [ ]:
def get_bboxes_and_scores_and_labels(ori_bboxes, class_names, n):
  bboxes = []
  scores = []
  labels = []
  for box in ori_bboxes:
    bbox = []
    score = []
    b_class = []
    for b in box:
      bbox.append(b[:4].tolist())
      score.append(b[4].tolist())
      b_class.append(class_names[n])
    bboxes.append(np.array(bbox))
    scores.append(np.array(score))
    labels.append(b_class)
  return bboxes,scores,labels

In [ ]:
class Args():
  def __init__(self, config, pkl_results, txt_path):
    self.config = config
    self.eval = 'bbox'
    self.format_only = False
    self.cfg_options = None
    self.eval_options = None
    self.txt_path = txt_path
    self.pkl_results = pkl_results

def eval_metric(args):
    cfg = Config.fromfile(args.config)
    assert args.eval or args.format_only, (
        'Please specify at least one operation (eval/format the results) with '
        'the argument "--eval", "--format-only"')
    if args.eval and args.format_only:
        raise ValueError('--eval and --format_only cannot be both specified')

    if args.cfg_options is not None:
        cfg.merge_from_dict(args.cfg_options)
    cfg.data.test.test_mode = True

    dataset = build_dataset(cfg.data.test)
    outputs = mmcv.load(args.pkl_results)

    kwargs = {} if args.eval_options is None else args.eval_options
    if args.format_only:
        dataset.format_results(outputs, **kwargs)
    if args.eval:
        eval_kwargs = cfg.get('evaluation', {}).copy()
        # hard-code way to remove EvalHook args
        for key in [
                'interval', 'tmpdir', 'start', 'gpu_collect', 'save_best',
                'rule', 'dynamic_intervals'
        ]:
            eval_kwargs.pop(key, None)
        eval_kwargs.update(dict(metric=args.eval, **kwargs))
        results = dataset.evaluate(outputs, **eval_kwargs)
        print(results)
        if args.txt_path:
          with open(args.txt_path, mode = 'a+') as f:
            f.write(str(results))

# Classes

In [ ]:
COCO_CLASSES = (
    "person",
    "bicycle",
    "car",
    "motorcycle",
    "airplane",
    "bus",
    "train",
    "truck",
    "boat",
    "traffic light",
    "fire hydrant",
    "stop sign",
    "parking meter",
    "bench",
    "bird",
    "cat",
    "dog",
    "horse",
    "sheep",
    "cow",
    "elephant",
    "bear",
    "zebra",
    "giraffe",
    "backpack",
    "umbrella",
    "handbag",
    "tie",
    "suitcase",
    "frisbee",
    "skis",
    "snowboard",
    "sports ball",
    "kite",
    "baseball bat",
    "baseball glove",
    "skateboard",
    "surfboard",
    "tennis racket",
    "bottle",
    "wine glass",
    "cup",
    "fork",
    "knife",
    "spoon",
    "bowl",
    "banana",
    "apple",
    "sandwich",
    "orange",
    "broccoli",
    "carrot",
    "hot dog",
    "pizza",
    "donut",
    "cake",
    "chair",
    "couch",
    "potted plant",
    "bed",
    "dining table",
    "toilet",
    "tv",
    "laptop",
    "mouse",
    "remote",
    "keyboard",
    "cell phone",
    "microwave",
    "oven",
    "toaster",
    "sink",
    "refrigerator",
    "book",
    "clock",
    "vase",
    "scissors",
    "teddy bear",
    "hair drier",
    "toothbrush",
)

# CFG

In [ ]:
class CFG():
  def __init__(self):

    self.method = "WBF"
    self.root = './wbf_output/'
    self.pkls = ['tood-1.pkl', 'cascade-10.pkl']
    self.weights = [1, 1]
    self.IMG_SIZE = 640
    self.iou_thr = 0.5
    self.iou_thr_2 = 0.7
    self.skip_box_thr = 0.0001
    self.skip_box_thr_2 = 0.0001
    self.txt_path = '/content/'+ '_'.join([pkl.split('.')[0] for pkl in self.pkls]) + '_result.txt'
    self.out_path = '_'.join([pkl.split('.')[0] for pkl in self.pkls])+'_'+str(self.iou_thr)+'_'+str(self.skip_box_thr)+'_'+self.method+'_ensem.pkl'
    print(self.prn_obj())

  # 输出所有属性
  def prn_obj(self):
    objs = '\n'.join(['  --%s:%s' % item for item in self.__dict__.items()])
    return objs
  
  # 修改属性的值
  def change_values(self, cfg_key, cfg_value):
    if cfg_key == 'pkls':
      self.pkls = cfg_value
    elif cfg_key == 'weights':
      self.weights = cfg_value
    elif cfg_key == 'IMG_SIZE':
      self.IMG_SIZE = cfg_value
    elif cfg_key == 'iou_thr':
      self.iou_thr = cfg_value
    elif cfg_key == 'skip_box_thr':
      self.skip_box_thr = cfg_value
    self.txt_path = '/content/'+ '_'.join([pkl.split('.')[0] for pkl in self.pkls]) + '_result.txt'
    self.out_path = '_'.join([pkl.split('.')[0] for pkl in self.pkls])+'_'+str(self.iou_thr)+'_'+str(self.skip_box_thr)+'_'+self.method+'_ensem.pkl'
  
  # 将属性写到txt文件中
  def write_cfgs(self):
    with open(cfg.txt_path, mode = 'a+') as f:
      f.write(self.prn_obj())

In [ ]:
cfg = CFG()

# Load pkl function

In [ ]:
def load_pkl():
  datas = []
  for pkl in cfg.pkls:
    data_path = cfg.root + pkl
    datas.append(np.array(mmcv.load(data_path)))
  print('pkls loaded')
  data1_df = pd.DataFrame(datas[0])
  print(data1_df.head(1))
  return datas

# Ensemble function

In [ ]:
def wbf_run(datas):
  class_num = 80
  img_num = 5000
  out_bboxes = []
  out_scores = []
  out_pkl = []

  ll = []
  ll2 = []
  for i in tqdm(range(class_num)):
    bboxes_ensem, scores_ensem, boxes_and_scores_ensem = [],[],[]
    bboxes, scores, labels = [],[],[]

    num_a = 0
    num_img_fusion = 0
    num_a2 = 0
    num_img_fusion2 = 0
    for data in datas:
      ori_bboxes = data[:,i]
      bbox, score, label = get_bboxes_and_scores_and_labels(ori_bboxes, COCO_CLASSES, i)
      bboxes.append(bbox)
      scores.append(score)
      labels.append(label)
    for j in range(img_num):
      boxes_list = []
      scores_list = []
      labels_list = []
      flag = 0
      pop_num = 0
      for k in range(len(cfg.pkls)):
        #若有框
        if len(bboxes[k][j]) > 0:
          boxes_list.append(bboxes[k][j])
          scores_list.append(scores[k][j])
          labels_list.append(labels[k][j])
          flag = 1
        
      boxes_e, scores_e, labels_e = [], [], []
      boxes_l, scores_l, labels_l = [], [], []
      if flag:
        if cfg.method == "WBF" and len(boxes_list) == 2:
          # for i in range(3):
          #   boxes_e1, scores_e1, labels_e1  = run_wbf(boxes_list, scores_list, labels_list, cfg.IMG_SIZE, 0.75, \
          #                                   cfg.skip_box_thr, cfg.weights)
          #   boxes_e2, scores_e2, labels_e2  = run_wbf(boxes_list, scores_list, labels_list, cfg.IMG_SIZE, 0.8, \
          #                                   cfg.skip_box_thr, cfg.weights)
          #   boxes_e3, scores_e3, labels_e3  = run_wbf(boxes_list, scores_list, labels_list, cfg.IMG_SIZE, 0.85, \
          #                                   cfg.skip_box_thr, cfg.weights)
          boxes_e, scores_e, labels_e, num_dui, num_dui2  = run_wbf(boxes_list, scores_list, labels_list, cfg.IMG_SIZE, cfg.iou_thr, \
                                          cfg.skip_box_thr, cfg.weights)
          num_img_fusion += 1
          num_a += num_dui
          if num_dui2 != 0:
            num_img_fusion2 += 1
            num_a2 += num_dui2

        else:
          boxes_e = boxes_list[0]
          scores_e = scores_list[0]
          labels_e = labels_list[0]

      #阈值在此处加

      box_and_score = np.array([]).reshape(0,5)
      if len(scores_e) > 0:
        scores_e = np.expand_dims(scores_e,axis = 1)
        box_and_score = np.concatenate((boxes_e,scores_e),axis = 1)

      bboxes_ensem.append(boxes_e)
      scores_ensem.append(scores_e)
      boxes_and_scores_ensem.append(box_and_score)

    ll.append(num_a / num_img_fusion)
    ll2.append(num_a2 / num_img_fusion2)
    out_bboxes.append(bboxes_ensem)
    out_scores.append(scores_ensem)
    out_pkl.append(boxes_and_scores_ensem)

  print('-----------------------------------------')
  print('-----------------------------------------')
  print(ll)
  print('-----------------------------------------')
  print(ll2)

  print("Average num of dui: ", sum(ll) / len(ll))
  print("Average num of dui that are more than 1: ", sum(ll2) / len(ll2))
  print("iou_thr: ", cfg.iou_thr)
  print("skip_box_thr: ", cfg.skip_box_thr)
  print('-----------------------------------------')
  print('-----------------------------------------')
  return out_pkl

# Run wbf and write pkl

In [ ]:
def prefilter_boxes(boxes, scores, labels, weights, thr):
    # Create dict with boxes stored by its label
    new_boxes = dict()

    for t in range(len(boxes)):

        if len(boxes[t]) != len(scores[t]):
            print('Error. Length of boxes arrays not equal to length of scores array: {} != {}'.format(len(boxes[t]), len(scores[t])))
            exit()

        if len(boxes[t]) != len(labels[t]):
            print('Error. Length of boxes arrays not equal to length of labels array: {} != {}'.format(len(boxes[t]), len(labels[t])))
            exit()

        for j in range(len(boxes[t])):
            score = scores[t][j]
            if score < thr:
                continue
            label = int(labels[t][j])
            box_part = boxes[t][j]
            x1 = float(box_part[0])
            y1 = float(box_part[1])
            x2 = float(box_part[2])
            y2 = float(box_part[3])

            # Box data checks
            if x2 < x1:
                warnings.warn('X2 < X1 value in box. Swap them.')
                x1, x2 = x2, x1
            if y2 < y1:
                warnings.warn('Y2 < Y1 value in box. Swap them.')
                y1, y2 = y2, y1
            if x1 < 0:
                warnings.warn('X1 < 0 in box. Set it to 0.')
                x1 = 0
            if x1 > 1:
                warnings.warn('X1 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                x1 = 1
            if x2 < 0:
                warnings.warn('X2 < 0 in box. Set it to 0.')
                x2 = 0
            if x2 > 1:
                warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                x2 = 1
            if y1 < 0:
                warnings.warn('Y1 < 0 in box. Set it to 0.')
                y1 = 0
            if y1 > 1:
                warnings.warn('Y1 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                y1 = 1
            if y2 < 0:
                warnings.warn('Y2 < 0 in box. Set it to 0.')
                y2 = 0
            if y2 > 1:
                warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                y2 = 1
            if (x2 - x1) * (y2 - y1) == 0.0:
                warnings.warn("Zero area box skipped: {}.".format(box_part))
                continue

            b = [int(label), float(score) * weights[t], x1, y1, x2, y2]
            if label not in new_boxes:
                new_boxes[label] = []
            new_boxes[label].append(b)
        # random.shuffle(new_boxes[label])

    # Sort each list in dict by score and transform it to numpy array
    for k in new_boxes:
        # random.shuffle(new_boxes[k])
        # print(type(new_boxes[k]))
        

       new_boxes[k] = np.array(new_boxes[k])
        
        # current_boxes = np.array(new_boxes[k])
        # new_boxes[k] = current_boxes[current_boxes[:, 1].argsort()[::-1]]
    # random.shuffle(new_boxes)

    return new_boxes

In [ ]:
def get_weighted_box(boxes, conf_type='avg'):
    """
    Create weighted box for set of boxes
    :param boxes: set of boxes to fuse
    :param conf_type: type of confidence one of 'avg' or 'max'
    :return: weighted box
    """
    # [int(label), float(score) * weights[t], x1, y1, x2, y2]
    # 长度为6
    box = np.zeros(6, dtype=np.float32)
    conf = 0
    conf_list = []
    
    # 输入一个box列表
    for b in boxes:
        # 分数加权的框位置
        box[2:] += (b[1] * b[2:])
        conf += b[1]
        conf_list.append(b[1])
    box[0] = boxes[0][0]
    if conf_type == 'avg':
        box[1] = conf / len(boxes)
    elif conf_type == 'max':
        box[1] = np.array(conf_list).max()
    box[2:] /= conf
    return box

In [ ]:
ious = []

@jit(nopython=True)
def bb_intersection_over_union(A, B) -> float:
    xA = max(A[0], B[0])
    yA = max(A[1], B[1])
    xB = min(A[2], B[2])
    yB = min(A[3], B[3])
    
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)

    if interArea == 0:
        return 0.0

    #如果完全包含，直接返回iou=0.8
    if (((A[0]>B[0])and(A[1]>B[1])and(A[2]<B[2])and(A[3]<B[3])) or ((A[0]<B[0])and(A[1]<B[1])and(A[2]>B[2])and(A[3]>B[3]))):
        return 0.8

    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (A[2] - A[0]) * (A[3] - A[1])
    boxBArea = (B[2] - B[0]) * (B[3] - B[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)
    # ious.append(iou)
    return iou

In [ ]:
def find_matching_box(boxes_list, new_box, match_iou):
    # match_iou is set to iou_thr
    # [int(label), float(score) * weights[t], x1, y1, x2, y2]
    best_iou = match_iou
    best_index = -1
    for i in range(len(boxes_list)):
    
        box = boxes_list[i]
        # label不一样就继续
        if box[0] != new_box[0]:
            continue
            
        # 算iou
        iou = bb_intersection_over_union(box[2:], new_box[2:])
        
        # 如果大于best，就计算为最大
        if iou > best_iou:
            best_index = i
            best_iou = iou
            
    # 找list里和box iou最大的框
    # index是框在list里的索引（位置）
    return best_index, best_iou

In [ ]:
def weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=None, iou_thr=0.55, skip_box_thr=0.0, conf_type='avg', allows_overflow=False):
    '''
    :param boxes_list: list of boxes predictions from each model, each box is 4 numbers.
    It has 3 dimensions (models_number, model_preds, 4)
    Order of boxes: x1, y1, x2, y2. We expect float normalized coordinates [0; 1]
    :param scores_list: list of scores for each model
    :param labels_list: list of labels for each model
    :param weights: list of weights for each model. Default: None, which means weight == 1 for each model
    :param iou_thr: IoU value for boxes to be a match
    :param skip_box_thr: exclude boxes with score lower than this variable
    :param conf_type: how to calculate confidence in weighted boxes. 'avg': average value, 'max': maximum value
    :param allows_overflow: false if we want confidence score not exceed 1.0

    :return: boxes: boxes coordinates (Order of boxes: x1, y1, x2, y2).
    :return: scores: confidence scores
    :return: labels: boxes labels
    '''

    if weights is None:
        weights = np.ones(len(boxes_list))
    if len(weights) != len(boxes_list):
        print('Warning: incorrect number of weights {}. Must be: {}. Set weights equal to 1.'.format(len(weights), len(boxes_list)))
        weights = np.ones(len(boxes_list))
    weights = np.array(weights)

    if conf_type not in ['avg', 'max']:
        print('Unknown conf_type: {}. Must be "avg" or "max"'.format(conf_type))
        exit()

    filtered_boxes = prefilter_boxes(boxes_list, scores_list, labels_list, weights, skip_box_thr)
    if len(filtered_boxes) == 0:
        return np.zeros((0, 4)), np.zeros((0,)), np.zeros((0,))

    overall_boxes = []

    num_all = 0   #该幅图像中每个堆框数量的平均
    num_all2 = 0   #该幅图像中框数量大于1的堆的框数量的平均
    for label in filtered_boxes:
        # 把label这个类别的取出来
        boxes = filtered_boxes[label]
        # random.shuffle(boxes)

        new_boxes = []
        weighted_boxes = []

        # Clusterize boxes
        # 遍历这一类全部的框
        for j in range(0, len(boxes)):
            # 把每个box都作为best box进行matching
            # weighted_boxes最开始是空的
            # list最开始是空的
            index, best_iou = find_matching_box(weighted_boxes, boxes[j], iou_thr)
            
            # 匹配指的是超过thr的最大iou的框
            if index != -1:
                # 如果找到了匹配，这个匹配在index位置，并且iou=best_iou
                # 把它放在new_boxes的第index个列表里
                # 相当于weighted_boxes中每一个box都对应new_boxes里一个列表，列表里有许多框
                new_boxes[index].append(boxes[j])
                # 把new_boxes这个列表里的box做加权位置
                weighted_boxes[index] = get_weighted_box(new_boxes[index], conf_type)
                # 作为最后的结果
            else:
                # 如果找不到匹配
                # 就把这个框放到new_boxes的最后，建立一个新的列表
                new_boxes.append([boxes[j].copy()])
                # 把这个框也放到weighted_boxes的最后
                weighted_boxes.append(boxes[j].copy())

        # Rescale confidence based on number of models and boxes
        dui = 0
        dui2 = 0
        num_dui2 = 0
        for i in range(len(new_boxes)):
            dui += len(new_boxes[i])
            if len(new_boxes[i]) > 1:
                dui2 += len(new_boxes[i])
                num_dui2 += 1
            if not allows_overflow:
                weighted_boxes[i][1] = weighted_boxes[i][1] * min(weights.sum(), len(new_boxes[i])) / weights.sum()
            else:
                weighted_boxes[i][1] = weighted_boxes[i][1] * len(new_boxes[i]) / weights.sum()
        num_all += dui / len(new_boxes)

        if num_dui2 > 0:
            num_all2 += dui2 / num_dui2
        
        overall_boxes.append(np.array(weighted_boxes))

    overall_boxes = np.concatenate(overall_boxes, axis=0)
    overall_boxes = overall_boxes[overall_boxes[:, 1].argsort()[::-1]]
    boxes = overall_boxes[:, 2:]
    scores = overall_boxes[:, 1]
    labels = overall_boxes[:, 0]
    return boxes, scores, labels, num_all, num_all2

In [ ]:
# out_pkl_df.iloc(0)[0]

In [ ]:
for i in range(100):
  cfg.iou_thr = 0.8 - i / 100
  cfg.skip_box_thr = 0.001
  print("iou_thr now: ", cfg.iou_thr)
  print("skip_box_thr now: ", cfg.skip_box_thr)

  datas = load_pkl()
  out_pkl = wbf_run(datas)
  out_pkl = list(map(list,zip(*out_pkl)))
  out_pkl_df = pd.DataFrame(out_pkl)
  # print(out_pkl_df.shape)
  # print(out_pkl_df.head(1))
  mmcv.dump(out_pkl, cfg.out_path)

  config = '/content/mmdetection/configs/yolox/yolox_l_8x8_300e_coco.py'
  pkl_results = '/content/' + cfg.out_path
  txt_path = cfg.txt_path
  args = Args(config, pkl_results, txt_path)
#   eval_metric(args)

iou_thr now:  0.8
skip_box_thr now:  0.001


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """


pkls loaded
                                                  0   1   2   3   4   5   6   \
0  [[387.52292, 67.35258, 499.34946, 345.60312, 0...  []  []  []  []  []  []   

   7   8   9   ...  70                                                 71  \
0  []  []  []  ...  []  [[495.87988, 199.22218, 614.37866, 229.37906, ...   

                                                  72  73  74  75  76  77  78  \
0  [[492.14343, 203.35875, 617.5813, 339.19812, 0...  []  []  []  []  []  []   

   79  
0  []  

[1 rows x 80 columns]


  0%|          | 0/80 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: X1 < 0 in box. Set it to 0.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: UserWarning: Y1 < 0 in box. Set it to 0.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Zero area box skipped: [0.         0.         0.         0.38597026].
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Zero area box skipped: [ 6.42253559e-01 -8.87132393e-07  6.42253559e-01  8.11952880e-02].
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Zero area box skipped: [0.54694836 0.81617691 0.54694836 0.86827333].
/usr/local/lib/p

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """


pkls loaded
                                                  0   1   2   3   4   5   6   \
0  [[387.52292, 67.35258, 499.34946, 345.60312, 0...  []  []  []  []  []  []   

   7   8   9   ...  70                                                 71  \
0  []  []  []  ...  []  [[495.87988, 199.22218, 614.37866, 229.37906, ...   

                                                  72  73  74  75  76  77  78  \
0  [[492.14343, 203.35875, 617.5813, 339.19812, 0...  []  []  []  []  []  []   

   79  
0  []  

[1 rows x 80 columns]


  0%|          | 0/80 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: X1 < 0 in box. Set it to 0.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: UserWarning: Y1 < 0 in box. Set it to 0.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Zero area box skipped: [0.         0.         0.         0.38597026].
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Zero area box skipped: [ 6.42253559e-01 -8.87132393e-07  6.42253559e-01  8.11952880e-02].
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Zero area box skipped: [0.54694836 0.81617691 0.54694836 0.86827333].
/usr/local/lib/p

KeyboardInterrupt: ignored

In [ ]:
import wandb

In [ ]:
# wandb.init(project="DWBF", entity="hsguo")
# for i in range(70, 100, 2):
#   cfg.iou_thr = 0.8
#   cfg.iou_thr_2 = i / 100
#   cfg.skip_box_thr = 0.001
#   cfg.skip_box_thr_2 = 0.001
#   print("iou_thr now: ", cfg.iou_thr)
#   print("iou_thr_2 now: ", cfg.iou_thr_2)
#   print("skip_box_thr now: ", cfg.skip_box_thr)
#   print("skip_box_thr_2 now: ", cfg.skip_box_thr_2)

#   datas = load_pkl()
#   out_pkl = wbf_run(datas)
#   out_pkl = list(map(list,zip(*out_pkl)))
#   out_pkl_df = pd.DataFrame(out_pkl)
#   # print(out_pkl_df.shape)
#   # print(out_pkl_df.head(1))
#   mmcv.dump(out_pkl, cfg.out_path)

#   config = '/content/mmdetection/configs/yolox/yolox_l_8x8_300e_coco.py'
#   pkl_results = '/content/' + cfg.out_path
#   txt_path = cfg.txt_path
#   args = Args(config, pkl_results, txt_path)
#   eval_metric(args)

# 测试脚本